In [1]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes import SearchIndexClient 
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    CorsOptions,
    SearchIndex,
    ScoringProfile,
    SearchFieldDataType,
    SimpleField,
    SearchableField
)

In [5]:
service_name = "snowflake-search-service"
admin_key = "gGdcNXgiHCwlU3Ifcrm2PRqtcs7ug3EvVmbVZnfuTfAzSeAF7mRp"

index_name = "snowflake"

#SDK Client
endpoint = "https://{}.search.windows.net/".format(service_name)
admin_client = SearchIndexClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

In [6]:
#Delete index it it exists:
try:
    result = admin_client.delete_index(index_name)
    print ('Index', index_name, 'Deleted')
except Exception as ex:
    print (ex)

<urllib3.connection.HTTPSConnection object at 0x00000208DB8BE880>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond


In [8]:
#Create index
name = index_name
fields=[
    #Do Snowflake tables have unique ID's? What can we use as a key?
    SearchableField(name="TableName", type=SearchFieldDataType.String, sortable=True),
    SearchableField(name="Comment", type=SearchFieldDataType.String, analyzer_name="en.lucene")]
    
    #Let's discuss the different indexes that we'll need to configure to search through the metadata.
    #Tag's might be a good metadata column to add because they can be included in the suggester

cors_options = CorsOptions(allowed_origins=["*"], max_age_in_seconds=60)
scoring_profiles = []
suggester = [{'name':'sg', 'source_fields': ["Comment"]}]  #"Tags"

In [9]:
index = SearchIndex(
    name=name,
    fields=fields,
    scoring_profiles=scoring_profiles,
    suggesters = suggester,
    cors_options=cors_options)

try:
    result = admin_client.create_index(index)
    print ('Index', result.name, 'created')
except Exception as ex:
    print (ex)

<urllib3.connection.HTTPSConnection object at 0x00000208DB8BE6A0>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond


### The next cell in the example notebook is a json document definition
* I believe this is the step that I need to connect the search service to Snowflake using the Python module

In [10]:
!pip install snowflake-connector-python

ERROR: Could not find a version that satisfies the requirement snowflake-connector-python (from versions: none)
ERROR: No matching distribution found for snowflake-connector-python
